# **Data Collection**

## Objectives

* Fetch data from Kaggle and download raw data.
* Check if any non-image files are downloaded and remove them.
* Split the dataset into test, train and validation sets.

## Inputs

* Kaggle JSON authentication token.
* Cherry leaves dataset from Kaggle.
* Generate inputs/cherry-leaves

## Outputs

The following folders will be generated:
* inputs/cherry_leaves_dataset/cherry-leaves/test/healthy
* inputs/cherry_leaves_dataset/cherry-leaves/test/powdery_mildew
* inputs/cherry_leaves_dataset/cherry-leaves/train/healthy
* inputs/cherry_leaves_dataset/cherry-leaves/train/powdery_mildew
* inputs/cherry_leaves_dataset/cherry-leaves/validation/healthy
* inputs/cherry_leaves_dataset/cherry-leaves/validation/powdery_mildew


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/project-5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/project-5'

# Install Kaggle

In [4]:
%pip install kaggle


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Set kaggle configuration to current working directory

---

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Get data from Kaggle

In [6]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves_dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 91%|██████████████████████████████████▌   | 50.0M/55.0M [00:02<00:00, 36.9MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 25.3MB/s]


Unzip folder

In [7]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Cleaning

Check if there are any non-image files and remove those that are not images

In [8]:
def remove_non_image_files(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # removes non-image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [9]:
remove_non_image_files(my_data_dir='inputs/cherry_leaves_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


## Split data into train, validation and test sets

In [10]:
import shutil
import random

def train_test_val_split(data_dir, train_set_ratio, val_set_ratio, test_set_ratio):

    if sum([train_set_ratio, val_set_ratio, test_set_ratio]) != 1.0:
        print("Sum of train_set_ratio, val_set_ratio and test_set_ratio should amount to 1.0")
        return
        
    # get labels/classes
    labels = os.listdir(data_dir)
    if 'test' in labels:
        pass
    else:
        # create train, validation and test folder with class labels as subfolder names
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=data_dir + '/' + folder + '/' + label)
                
        for label in labels:

            files = os.listdir(data_dir + '/' + label)
            random.shuffle(files)
            
            train_set_qty = int(len(files) * train_set_ratio)
            val_set_qty = int(len(files) * val_set_ratio)
            
            count = 1
            
            for file_name in files:
            
                if count <= train_set_qty:
                    # move given file to train set
                    shutil.move(data_dir + '/' + label + '/' + file_name,
                                data_dir + '/train/' + label + '/' + file_name)
                
                elif count <= (sum([train_set_qty, val_set_qty])):
                    # move given file to validation set 
                    shutil.move(data_dir + '/' + label + '/' + file_name,
                                data_dir + '/validation/' + label + '/' + file_name)
                else:
                    # move remaining files to test
                    shutil.move(data_dir + '/' + label + '/' + file_name,
                                data_dir + '/test/' + label + '/' + file_name)
                
                count += 1
                
            os.rmdir(data_dir + '/' + label)

    

In [11]:
train_test_val_split(data_dir=f"inputs/cherry_leaves_dataset/cherry-leaves", train_set_ratio=0.7,
                    val_set_ratio=0.1, test_set_ratio=0.2)

### Check number of files in each set

In [12]:
file_count, data_split, group = [], [], []
base_dir = "inputs/cherry_leaves_dataset/cherry-leaves" 

for label in ['healthy', 'powdery_mildew']:
    for folder in ['train', 'test', 'validation']:
        file_path = os.path.join(base_dir, folder, label)
        files = os.listdir(file_path)
        num_files = len(files)
        file_count.append(num_files)
        data_split.append(folder)
        group.append(label)
        print(f'* {folder} - {label}: {len(os.listdir(file_path))} images')
        

* train - healthy: 1472 images
* test - healthy: 422 images
* validation - healthy: 210 images
* train - powdery_mildew: 1472 images
* test - powdery_mildew: 422 images
* validation - powdery_mildew: 210 images
